Diego Toribio  
Professor Eilin Perez  
SS-322: History and Visuality  
Final Project

In [ ]:
import ee
import geemap
from IPython.display import Image

# Authenticate & initialize
ee.Authenticate()
ee.Initialize(project='toribiodiego-ece471')